experimento GA.01 - senha de tamanho variavel
========================================



## Introdução



Delete este texto e escreva uma breve introdução sobre seu experimento.



## Objetivo



**Objetivo**: Resolver o problema da senha sem fornecer a informação do tamanho da senha para a função que gera a população.

**Dica**: A função objetivo terá que quantificar em sua métrica tanto se o candidato acertou as letras quanto se acertou o tamanho da senha.

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
from funcoes import populacao_inicial_senha_variavel
from funcoes import funcao_objetivo_pop_senha_variavel
from funcoes import selecao_torneio_min
from funcoes import cruzamento_ponto_senha_variavel as funcao_cruzamento
from funcoes import mutacao_senha_variavel
from funcoes import gene_letra
import random

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 10
NUM_GERACOES = 2000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 2

# relacionadas ao problema a ser resulvido
MAXIMO = 100
PESO_PENALIDADE = 60
SENHA = "isalindadms" # o que estou tentando descobrir
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz" # todas letras que podem estar na minha senha
# NUM_GENES = len(SENHA)

In [3]:
# funções locais

def cria_populacao_inicial(tamanho, tamanho_senha):
    return populacao_inicial_senha_variavel(TAMANHO_POP, MAXIMO, LETRAS_POSSIVEIS) #usa um tamanho máximo chute 

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_senha_variavel(populacao, SENHA , PESO_PENALIDADE)

def funcao_selecao(populacao, fitness):
    return selecao_torneio_min(populacao, fitness, NUM_COMBATENTES_NO_TORNEIO)

def funcao_mutacao(individuo):
    return mutacao_senha_variavel(individuo, LETRAS_POSSIVEIS, MAXIMO)

In [5]:
populacao = populacao_inicial_senha_variavel(TAMANHO_POP, MAXIMO, LETRAS_POSSIVEIS)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python
                        # assim, o primeiro indivíduo já substituirá, 
                        # pois vai ser melhor que infinito

print("Progresso da melhor senha já vista:")

while melhor_fitness_ja_visto != 0:
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2] # os pais vão pares (em númros cardinais)
    maes = populacao[1::2] # as mães serão ímpares
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness
        
        print("".join(melhor_individuo_ja_visto), "- fitness:", melhor_fitness_ja_visto)

print()
print("Melhor palpite da senha encontrado:")
print("".join(melhor_individuo_ja_visto))

Progresso da melhor senha já vista:
quebudgnq - fitness: 195
tmoaspvltfx - fitness: 111
qfelnfikhho - fitness: 66
tmomokahhko - fitness: 61
tmomomahhko - fitness: 59
imomahhhkkt - fitness: 56
imhhmhhkkko - fitness: 54
imomhkkkkku - fitness: 53
imhmmmhkkko - fitness: 46
ihiohmnberq - fitness: 43
qmaemiiaems - fitness: 36
mqbqbqbbdmi - fitness: 35
irmmimgcggv - fitness: 32
gufjfnffdfq - fitness: 30
ltdlhhdddlp - fitness: 21
imcmmmcccmt - fitness: 19
hrdjjrdadmy - fitness: 18


KeyboardInterrupt: 

-   Use células de código para o código.

-   Use células de texto para a discussão.

-   A discussão não deve ser feita em comentários dentro das células de código. Toda discussão deve acontecer após o resultado sendo discutido foi apresentado. Exemplo: não discuta um gráfico antes de apresentá-lo.



## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [ ]:
caracteres = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "z"]

for caractere in caracteres:
    numero = ord(caractere)
    print(f"O caractere '{caractere}' converte para o número {numero}")

In [ ]:
individuo_senha_tv(LETRAS_POSSIVEIS, SENHA)